In [ ]:
import json

all_tools = set()
with open("/home/snrobot/lin/ComplexFuncBench/data/ComplexFuncBench.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)
        id = data["id"]
        category = id.split("_")[0]
        funcs = data["functions"]
        for tool in funcs:
            all_tools.add(json.dumps(tool))

output_data = {'function': [json.loads(tool) for tool in all_tools]}
with open("/home/snrobot/lin/GraphLinkTools/Tools/All_Tools.json", 'w', encoding='utf-8') as outfile:
    json.dump(output_data, outfile, ensure_ascii=False, indent=4)
print(len(all_tools))

In [ ]:
import json

unique_tools = set()

with open("/home/snrobot/lin/GraphLinkTools/Tools/All_Tools.json", 'r', encoding='utf-8') as f:
    data = json.load(f)
    tools = data['function']
    for tool in tools:
        unique_tools.add(tool["name"])
print(unique_tools)
print(len(unique_tools))

        
unique_tools_rapidapi = set()
with open("/home/snrobot/lin/ComplexFuncBench/utils/tool_info.json", 'r', encoding='utf-8') as f:
    tool_info = json.load(f)
    tools = tool_info["booking-com15"]['name_to_url']
    unique_tools_rapidapi = set(tools.keys())
print(len(unique_tools_rapidapi))
print(unique_tools_rapidapi - unique_tools)
    



{'Search_Attraction_Location', 'Get_Hotel_Reviews(Tips)', 'Search_Hotels_By_Coordinates', 'Get_Hotel_Reviews(Tips)_Sort_Option', 'Search_Hotel_Destination', 'Get_Seat_Map', 'Taxi_Search_Location', 'Get_Flight_Details', 'Get_Min_Price', 'Vehicle_Supplier_Review', 'Payment_features_of_the_Hotel', 'Property_Children_Policies', 'Get_Hotel_Review_Scores', 'Get_Question_And_Answer', 'Get_Attraction_Details', 'Get_Hotel_Details', 'Vehicle_Details', 'Get_Packages', 'Booking_Summary', 'Search_Flight_Location', 'Get_Description_And_Info', 'Get_Availability', 'Location_to_Lat_Long', 'Get_Room_Availability', 'Search_Attractions', 'Search_Flights_Multi_Stops', 'Search_Car_Rentals', 'Get_Room_List_With_Availability', 'Search_Flights', 'Vehicle_Supplier_Details', 'Vehicle_Supplier_Ratings', 'Get_Filter', 'Search_Car_Location', 'Search_Taxi', 'Get_Availability_Calendar', 'Get_Min_Price_Multi_Stops', 'Get_Hotel_Policies', 'Search_Hotels', 'Get_Popular_Attraction_Near_By', 'Get_Nearby_Cities'}
40
48
{'G

In [21]:
import json

Apis = {}
with open("/home/snrobot/lin/GraphLinkTools/Tools/Apis.json", 'r', encoding='utf-8') as f:
    data = json.load(f)
    Apis = data['functions']
    
with open("/home/snrobot/lin/GraphLinkTools/Tools/All_Tools.json", 'r', encoding='utf-8') as f:
    data = json.load(f)
    Tools = data['function']
    
tool_descriptions = {tool["name"]: tool["description"].split("EndPoint: ")[0] for tool in Tools}
tool_endpoint = {tool["name"]: tool["description"].split("EndPoint: ")[1] for tool in Tools}

for api in Apis:
    tool_name = api["name"]
    if tool_name in tool_descriptions:
        api["func_description"] = tool_descriptions[tool_name]
        api["endpoint"] = tool_endpoint[tool_name]
    
funcs = {"functions":Apis}
with open("/home/snrobot/lin/GraphLinkTools/Tools/Apis.json", 'w', encoding='utf-8') as f:
    json.dump(funcs, f, ensure_ascii=False, indent=4)
    
    

In [22]:
with open("/home/snrobot/lin/ComplexFuncBench/utils/tool_info.json", 'r', encoding='utf-8') as f:
    tool_info = json.load(f)
    tools = tool_info["booking-com15"]['name_to_url']

for key in tools:
    url = tools[key]
    last_part = url.split('https://booking-com15.p.rapidapi.com')[-1]  # 按 '/' 分割并取最后一段
    tools[key] = last_part

Apis = {}
with open("/home/snrobot/lin/GraphLinkTools/Tools/Apis.json", 'r', encoding='utf-8') as f:
    data = json.load(f)
    Apis = data['functions']
    
for api in Apis:
    tool_name = api["name"]
    if tool_name in tools:
        api["endpoint"] = tools[tool_name]
        
funcs = {"functions":Apis}
with open("/home/snrobot/lin/GraphLinkTools/Tools/Apis(1).json", 'w', encoding='utf-8') as f:
    json.dump(funcs, f, ensure_ascii=False, indent=4)





In [ ]:
import json

input_file = "/home/snrobot/lin/ComplexFuncBench/data/ComplexFuncBench.jsonl"
output_file = "/home/snrobot/lin/GraphLinkTools/Tools/Function_observation.json"


with open(input_file, "r") as f:
    datas = [json.loads(line) for line in f]
    
results = []
for data in data:
    conversations = data["conversations"]
    i = 0
    while i < len(conversations):
        if conversations[i]["role"] == "assistant" and "function_call" in conversations[i]:
            calls = conversations[i]["function_call"]
            j = i + 1
            while j < len(conversations):
                if conversations[j]["role"] == "observation":
                        observations = conversations[j]["content"]
                        break
                j += 1
            else:
                observations = None
            
            for call, obs in zip(calls, observations):
                results.append({
                    "function_name": call["name"],
                    "arguments": call["arguments"],
                    "observation": obs
                }) 
        i += 1
with open(output_file, 'w', encoding='utf-8') as out_f:
    json.dump(results, out_f, ensure_ascii=False, indent=4)

print(f"共提取 {len(results)} 条 function_call 与 observation 映射，已保存至 {output_file}")
    
